In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from operator import itemgetter

In [2]:
path = "C:\\Users\\Tiziano Pacifico\\Desktop\\TESI General\\TESI_FINAL_STAGE\\FASE 6\\DFs\\"
filename = "final_df.json"
df_in = pd.read_json(path+filename, lines=True)

In [3]:
#Divido il dataframe per intervalli settimanali, riduco le colonne a quelle che mi initeressano
df_in['time_utc'] = pd.to_datetime(df_in['utc_datetime_str'])
df_small = df_in[['time_utc','GPT_label','polarity','text']].copy()
day_intervals = pd.date_range(start=df_small['time_utc'].min(), end=df_small['time_utc'].max(), freq='1D')

day_df_list = []
for i in range(len(day_intervals) - 1):
    start_date = day_intervals[i]
    end_date = day_intervals[i + 1]
    week_data = df_small[(df_small['time_utc'] >= start_date) & (df_small['time_utc'] < end_date)].copy()
    day_df_list.append(week_data)
day_df_list.append(df_small[(df_small['time_utc'] >= end_date) & (df_small['time_utc'] <= df_small['time_utc'].max())].copy())

In [4]:
topics = list(df_small['GPT_label'].unique())

In [5]:
#ottengo un dizionario in cui le chiavi sono i topic e i valori sono liste di DFs. Ogni lista contiene i DFs per quei topic divisi per giorni
dfs_per_days_and_topics = {}
for topic in topics:
    days_df_list_per_topic = []
    for dayly_df in day_df_list:
        days_df_list_per_topic.append(dayly_df[dayly_df['GPT_label'] == topic])
    dfs_per_days_and_topics[topic] = days_df_list_per_topic

In [6]:
avg_post_per_days_per_topic_list = {}
for topic in topics:
    avg_post_per_day = (df_small.groupby('GPT_label')['GPT_label'].count()[topic])/len(dfs_per_days_and_topics[topic])
    avg_post_per_days_per_topic_list[topic] = round(avg_post_per_day)

In [7]:
filename = "count_series.json"
count_series = pd.read_json(path+filename)
count_series = count_series.drop('name', axis=1).rename(columns={'index':'topic'})


In [8]:
period_max_activity = {}
i = 0
for topic in topics:
    #print(i)
    
    start = False
    period_max_activity[topic] = []
    start_date = None
    stop_date = None
    j = 0
    for df_per_day_and_topic in dfs_per_days_and_topics[topic]:
        if(df_per_day_and_topic.shape[0] > avg_post_per_days_per_topic_list[topic] and not(start)):
            start = True
            start_date = df_per_day_and_topic['time_utc'].iloc[0]
            stop_date = None
            #print(f"start: {start_date}")
        #print(f"_____{df_per_day_and_topic.shape[0]}")
        if(df_per_day_and_topic.shape[0] < avg_post_per_days_per_topic_list[topic] and start):
            start = False
            if(df_per_day_and_topic.shape[0] <= 0):
                stop_date = day_intervals[j]
                #print(f"stop: {stop_date}")
            else:
                stop_date = df_per_day_and_topic['time_utc'].iloc[-1]
            
        if(start_date != None and stop_date != None):
            period_max_activity[topic].append((start_date, stop_date))
            start_date = None
            stop_date = None
        j += 1
    if(start_date != None and stop_date == None):
        stop_date = dfs_per_days_and_topics[topics[4]][-1]['time_utc'].iloc[-1]
        period_max_activity[topic].append((start_date, stop_date))
    i += 1
#for topic in topics:
#    for p in period_max_activity[topic]:
#        print(p[0],p[1])
#    print("-------------------")
        
    

In [9]:
colori = [
    "rgb(255, 0, 0)",    # Rosso
    "rgb(0, 255, 0)",    # Verde
    "rgb(0, 0, 255)",    # Blu
    "rgb(255, 255, 0)",  # Giallo
    "rgb(255, 0, 255)",  # Magenta
    "rgb(0, 255, 255)",  # Ciano
    "rgb(128, 0, 0)",    # Marrone
    "rgb(128, 128, 0)",  # Oliva
    "rgb(0, 128, 0)",    # Verde Scuro
    "rgb(128, 0, 128)",  # Viola
    "rgb(0, 128, 128)",  # Verde Petrolio
    "rgb(0, 0, 128)",    # Blu Marino
    "rgb(70, 130, 180)", # Blu Acciaio
    "rgb(220, 20, 60)",  # Rosso Cremisi
    "rgb(255, 140, 0)",  # Arancione Scuro
    "rgb(46, 139, 87)",  # Verde Mare
    "rgb(255, 69, 0)",   # Rosso Arancio
    "rgb(75, 0, 130)",   # Indaco
    "rgb(240, 230, 140)",# Khaki
    "rgb(106, 90, 205)", # Lavanda
    "rgb(173, 216, 230)",# Azzurro
    "rgb(255, 105, 180)" # Rosa Caldo
]
color_map = {topic: colori[idx % len(colori)] for idx, topic in enumerate(period_max_activity.keys())}

In [10]:
count_series['period_max_Activity'] = count_series['topic'].map(period_max_activity)

In [11]:
percentage_days_max_activity_per_topic = {}
for topic in topics:
    n_days_activity = 0
    for start,end in period_max_activity[topic]:
        if start.date()!= end.date():
            n_days_activity += (end.date()-start.date()).days
        else:
            n_days_activity += 1
    percentage_days_max_activity_per_topic[topic] = str(round(n_days_activity/len(day_intervals)*100)) + "_%"
count_series['percentage_max_Activity'] = count_series['topic'].map(percentage_days_max_activity_per_topic)

In [12]:
fig = go.Figure()
y_values = {row['topic']: idx for idx, row in count_series.iterrows()}

for index,row in count_series.iterrows():
    for start, end in period_max_activity[row['topic']]:
        fig.add_trace(go.Scatter(
            x=[start, end],
            y=[y_values[row['topic']], y_values[row['topic']]],
            mode="lines",
            line=dict(color=color_map[row['topic']],width=8),
            name=topic
        ))

fig.update_layout(
    title="Picchi di Attività per Topic in ordine ascendente di volume",
    xaxis_title="Data",
    yaxis_title="Topic",
    yaxis=dict(
        tickmode='array',
        tickvals=list(y_values.values()),
        ticktext=list(y_values.keys()),
        tickfont=dict(
            size=14,  # Aumenta la dimensione del font delle label dell'asse Y
            color='black'  # Opzionale: cambia il colore del font (esempio)
        )
    ),
    showlegend=False,  # Nasconde la leggenda
    height=1000
)


for index,row in count_series.iterrows():
    fig.add_annotation(
        text=row['percentage_max_Activity'],
        xref="paper",  # Posizione orizzontale relativa al contenuto del grafico
        x=1.07,  # Regola questa coordinata per posizionare l'etichetta
        y=y_values[row['topic']],  # L'indice y associato all'etichetta
        font=dict(size=14, color='black'),
        showarrow=False,  # Nascondi la freccia di riferimento
    )
# Mostra il grafico
fig.show()

In [13]:
period_activity = {}
i = 0
for topic in topics:
    #print(i)
    
    start = False
    period_activity[topic] = []
    start_date = None
    stop_date = None
    j = 0
    for df_per_day_and_topic in dfs_per_days_and_topics[topic]:
        if(df_per_day_and_topic.shape[0] > 0 and not(start)):
            start = True
            start_date = df_per_day_and_topic['time_utc'].iloc[0]
            stop_date = None
            #print(f"start: {start_date}")
        #print(f"_____{df_per_day_and_topic.shape[0]}")
        if(df_per_day_and_topic.shape[0] <= 0 and start):
            start = False
            if(df_per_day_and_topic.shape[0] <= 0):
                stop_date = day_intervals[j]
                #print(f"stop: {stop_date}")
            else:
                stop_date = df_per_day_and_topic['time_utc'].iloc[-1]
            
        if(start_date != None and stop_date != None):
            period_activity[topic].append((start_date, stop_date))
            start_date = None
            stop_date = None
        j += 1
    if(start_date != None and stop_date == None):
        stop_date = dfs_per_days_and_topics[topics[4]][-1]['time_utc'].iloc[-1]
        period_activity[topic].append((start_date, stop_date))
    i += 1

In [14]:
fig = go.Figure()
y_values = {topic: idx for idx, topic in enumerate(period_activity.keys())}

for topic, periods in period_activity.items():
    for start, end in periods:
        fig.add_trace(go.Scatter(
            x=[start, end],
            y=[y_values[topic], y_values[topic]],
            mode="lines",
            line=dict(color=color_map[topic],width=8),
            name=topic
        ))

fig.update_layout(
    title="Periodi di Attività per Topic",
    xaxis_title="Data",
    yaxis_title="Topic",
    yaxis=dict(
        tickmode='array',
        tickvals=list(y_values.values()),
        ticktext=list(y_values.keys()),
                tickfont=dict(
            size=14,  # Aumenta la dimensione del font delle label dell'asse Y
            color='black'  # Opzionale: cambia il colore del font (esempio)
        )
    ),
    
    showlegend=False,  # Nasconde la leggenda
    height=1000
)

# Mostra il grafico
fig.show()

In [15]:
count_series['period_Activity'] = count_series['topic'].map(period_activity)

In [16]:
percentage_days_activity_per_topic = {}
for topic in topics:
    n_days_activity = 0
    for start,end in period_activity[topic]:
        if start.date()!= end.date():
            n_days_activity += (end.date()-start.date()).days
        else:
            n_days_activity += 1
    percentage_days_activity_per_topic[topic] = str(round(n_days_activity/len(day_intervals)*100)) + "_%"
count_series['percentage_Activity'] = count_series['topic'].map(percentage_days_activity_per_topic)

In [17]:
fig = go.Figure()
y_values = {row['topic']: idx for idx, row in count_series.iterrows()}

for index,row in count_series.iterrows():
    for start, end in period_activity[row['topic']]:
        fig.add_trace(go.Scatter(
            x=[start, end],
            y=[y_values[row['topic']], y_values[row['topic']]],
            mode="lines",
            line=dict(color=color_map[row['topic']],width=8),
            name=topic
        ))

fig.update_layout(
    title="Periodi di Attività per Topic in ordine ascendente di volume",
    xaxis_title="Data",
    yaxis_title="Topic",
    yaxis=dict(
        tickmode='array',
        tickvals=list(y_values.values()),
        ticktext=list(y_values.keys()),
                tickfont=dict(
            size=14,  # Aumenta la dimensione del font delle label dell'asse Y
            color='black'  # Opzionale: cambia il colore del font (esempio)
        )
    ),
    
    showlegend=False,  # Nasconde la leggenda
    height=1000
)
for index,row in count_series.iterrows():
    fig.add_annotation(
        text=row['percentage_Activity'],
        xref="paper",  # Posizione orizzontale relativa al contenuto del grafico
        x=1.07,  # Regola questa coordinata per posizionare l'etichetta
        y=y_values[row['topic']],  # L'indice y associato all'etichetta
        font=dict(size=14, color='black'),
        showarrow=False,  # Nascondi la freccia di riferimento
    )
# Mostra il grafico
fig.show()

In [20]:
df_small.head()

,time_utc,GPT_label,polarity,text
0,2023-05-01 17:00:56,AI Chatbot Interaction and Assistance on Cloud...,0.117045,ChatGPT does not do stand up comedyI don’t thi...
1,2023-05-01 16:59:32,Chatbot Interaction and Usage on Discord,0.158333,Dreamcatcher - ChatGPT Retrieval on SteroidsA ...
2,2023-05-01 16:57:39,Dan Discussing ChatGPT Jailbreaking Ethics,0.166667,"Thoughts on this?[How to Jailbreak ChatGPT, Li..."
3,2023-05-01 16:50:02,AI and ChatGPT in Everyday Life,-0.074985,Emergent abilities of chat GPT. We don't reall...
4,2023-05-01 16:43:19,AI Chatbot Interaction and Assistance on Cloud...,0.200000,This chatgpt app gives me a 20 free chat credi...


In [18]:
period_not_max = {}
for topic,periods in period_activity.items():
    period_not_max[topic] = []
    for start,end in periods:
        for start1,end1 in period_max_activity[topic]:
            if (start <= start1 and end >= end1):
                period_not_max[topic].append((start,start1))
                period_not_max[topic].append((end1,end))
            else:
                period_not_max[topic].append((start,end))


In [19]:
std_per_topic_activity = {}
for topic,periods in period_activity.items():
    df_vuoto = pd.DataFrame(columns=df_small.columns)
    for start,end in periods:
        df_vuoto = pd.concat([df_vuoto, df_small[(df_small['time_utc'] >= start) & (df_small['time_utc'] <= end) & (df_small['GPT_label'] == topic)]])
    std_per_topic_activity[topic] = df_vuoto['polarity'].std()


In [20]:
std_per_topic_max = {}
for topic,periods in period_max_activity.items():
    df_vuoto = pd.DataFrame(columns=df_small.columns)
    for start,end in periods:
        df_vuoto = pd.concat([df_vuoto, df_small[(df_small['time_utc'] >= start) & (df_small['time_utc'] <= end) & (df_small['GPT_label'] == topic)]])
    std_per_topic_max[topic] = df_vuoto['polarity'].std()

In [21]:
std_per_topic_not_max = {}
for topic,periods in period_not_max.items():
    df_vuoto = pd.DataFrame(columns=df_small.columns)
    for start,end in periods:
        df_vuoto = pd.concat([df_vuoto, df_small[(df_small['time_utc'] >= start) & (df_small['time_utc'] <= end) & (df_small['GPT_label'] == topic)]])
    std_per_topic_not_max[topic] = df_vuoto['polarity'].std()

In [22]:
count_series.head()

,topic,data,period_max_Activity,percentage_max_Activity,period_Activity,percentage_Activity
0,AI and ChatGPT in Everyday Life,13263,"[(2022-12-07 20:03:59, 2022-12-18 20:32:58), (...",42_%,"[(2022-12-02 14:26:01, 2023-04-30 21:08:43)]",99_%
1,Chatbot Interaction and Usage on Discord,8155,"[(2022-12-08 18:43:20, 2022-12-17 20:21:29), (...",34_%,"[(2022-12-02 19:23:24, 2023-04-30 21:08:43)]",99_%
2,Discord Server Chatbot Interaction and User Re...,7704,"[(2022-12-20 19:42:40, 2022-12-21 20:43:04), (...",34_%,"[(2022-12-04 23:33:58, 2022-12-05 20:19:28), (...",69_%
3,AI Chatbot Interaction and Response automation,5197,"[(2023-03-20 20:00:44, 2023-04-22 20:32:54), (...",25_%,"[(2022-12-02 14:18:59, 2023-04-30 21:08:43)]",99_%
4,Bot Moderation in Online Comment Sections,4714,"[(2023-01-26 15:38:08, 2023-01-27 16:49:02), (...",14_%,"[(2022-12-08 11:44:33, 2022-12-09 20:19:28), (...",36_%


In [23]:
count_series['std_per_topic_max'] = count_series['topic'].map(std_per_topic_max)
count_series['std_per_topic_not_max'] = count_series['topic'].map(std_per_topic_not_max)

In [24]:
fig = go.Figure()
fig.add_trace(go.Bar(
    y=list(list(count_series['topic'])),
    x=list(list(count_series['std_per_topic_not_max'])),
    name='not max',
    orientation='h', 
    marker=dict(
        color='rgba(246, 78, 139, 0.6)',
        line=dict(color='rgba(246, 78, 139, 1.0)', width=3)
    )
))
fig.add_trace(go.Bar(
    y=list(list(count_series['topic'])),
    x=list(list(count_series['std_per_topic_max'])),
    name='max',
    orientation='h',
    marker=dict(
        color='rgba(58, 71, 80, 0.6)',
        line=dict(color='rgba(58, 71, 80, 1.0)', width=3)
    )
))
fig.update_layout(
    legend=dict(
        title="Legenda",  # Titolo della legenda
        yanchor="top",  # Posizione verticale della legenda
        y=1.0,  # Posizione verticale della legenda
        xanchor="left",  # Posizione orizzontale della legenda
        x=1  # Posizione orizzontale della legenda
    ),
    title=dict(
        text='<b>Confronto tra std nei periodi di massima attività, rispetto ai periodi di attività regolare',  # Title text
        x=0.5,  # x=0.5 centers the title
        xanchor='center',  # 'center' ensures the title is centered
        font=dict(
            size=24,
            color='black',
            family="Arial, sans-serif",
        )
        )
)


fig.update_layout(
    width=1000,   # Larghezza in pixel
    height=600   # Altezza in pixel
)



In [25]:
count = 0
for topic in topics:
    if(std_per_topic_not_max[topic] > std_per_topic_max[topic]):
        count += 1
print(count)

14


In [26]:
std_difference_per_topic = {}
for topic in topics:
    std_difference_per_topic[topic] = std_per_topic_not_max[topic] - std_per_topic_max[topic]
    

In [27]:
dizionario_ordinato = dict(sorted(std_difference_per_topic.items(), key=itemgetter(1), reverse=True))